In [ ]:
!git clone https://github.com/microsoft/unilm.git
!cd unilm && git reset --hard 1111fee

Cloning into 'unilm'...
remote: Enumerating objects: 7541, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 7541 (delta 91), reused 184 (delta 73), pack-reused 7317
Receiving objects: 100% (7541/7541), 24.94 MiB | 28.28 MiB/s, done.
Resolving deltas: 100% (3170/3170), done.
HEAD is now at 1111fee Update gradio_app.py


In [ ]:
%cd /content/unilm/beit3

/content/unilm/beit3


In [ ]:
%%capture
!sed -i 's/torch._six/torch/g' utils.py
!sed -i 's/torchmetrics==0.7.3/torchmetrics==0.11.4/g' requirements.txt

In [ ]:
!pip install -qr requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.6/444.6 kB 36.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 5.9 MB/s eta 0:0

In [ ]:
!wget https://conversationhub.blob.core.windows.net/beit-share-public/beit3/pretraining/beit3_base_patch16_224.pth
!wget https://conversationhub.blob.core.windows.net/beit-share-public/beit3/sentencepiece/beit3.spm

--2023-06-05 00:57:11--  https://conversationhub.blob.core.windows.net/beit-share-public/beit3/pretraining/beit3_base_patch16_224.pth
Resolving conversationhub.blob.core.windows.net (conversationhub.blob.core.windows.net)... 52.191.176.36
Connecting to conversationhub.blob.core.windows.net (conversationhub.blob.core.windows.net)|52.191.176.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 554351015 (529M) [application/zip]
Saving to: ‘beit3_base_patch16_224.pth’

beit3_base_patch16_ 100%[===================>] 528.67M  27.8MB/s    in 23s     

2023-06-05 00:57:35 (22.8 MB/s) - ‘beit3_base_patch16_224.pth’ saved [554351015/554351015]

--2023-06-05 00:57:35--  https://conversationhub.blob.core.windows.net/beit-share-public/beit3/sentencepiece/beit3.spm
Resolving conversationhub.blob.core.windows.net (conversationhub.blob.core.windows.net)... 52.191.176.36
Connecting to conversationhub.blob.core.windows.net (conversationhub.blob.core.windows.net)|52.191.176.36|:4

In [ ]:
%%capture

import sys
from argparse import Namespace

import torch
from torch import nn

import utils
from datasets import build_transform
from modeling_utils import BEiT3Wrapper, _get_base_config
from run_beit3_finetuning import get_args


class BEiT3BinaryImageClassification(BEiT3Wrapper):
    def __init__(self, args_: Namespace):
        super().__init__(args=args_)
        embed_dim = args_.encoder_embed_dim
        self.fc_norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, 1)

        self.fc_norm.apply(self._init_weights)
        self.head.apply(self._init_weights)
        init_scale = 0.001
        if isinstance(self.head, nn.Linear):
            self.head.weight.data.mul_(init_scale)
            self.head.bias.data.mul_(init_scale)

    def forward(self, image):
        with torch.no_grad(), torch.cuda.amp.autocast():
            x = self.beit3(textual_tokens=None, visual_tokens=image)['encoder_out']
        t = x[:, 1:, :]
        cls_x = self.fc_norm(t.mean(1))
        return self.head(cls_x)


sys.argv = [
    'script.py',
    '--model', 'beit3_base_patch16_224',
    '--input_size', '224',
    '--task', 'flickr30k',  # just because required
    '--batch_size', '16',
    '--sentencepiece_model', '/content/unilm/beit3/beit3.spm',
    '--finetune', '/content/unilm/beit3/beit3_base_patch16_224.pth',
    '--num_workers', '0',
    '--randaug',  # augmentations
]
args = get_args()[0]

beit_train_transform = build_transform(is_train=True, args=args)
beit_eval_transform = build_transform(is_train=False, args=args)

beit_args = _get_base_config(**vars(args))
beit_wrapper = BEiT3BinaryImageClassification(beit_args)
utils.load_model_and_may_interpolate(args.finetune, beit_wrapper, args.model_key, args.model_prefix)
beit_wrapper.to(args.device)
beit_wrapper.beit3.eval()

In [ ]:
%%capture
!pip install git+https://github.com/microsoft/torchscale@2b10135
!pip install -qU timm pytorch_lightning

In [ ]:
# https://www.kaggle.com/settings/account > "Create New Token"
from google.colab import files
files.upload()
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/kaggle.json
!kaggle datasets download -d dmitryvinnik/artifact-60k-balanced
!kaggle datasets download -d dmitryvinnik/rd-predict-dataset
!kaggle datasets download -d dmitryvinnik/beit3-fake-image-detector
!unzip -q artifact-60k-balanced.zip -d ./artifact_60k_balanced
!mkdir rd_predict_dataset
!unzip -q rd-predict-dataset.zip -d ./rd_predict_dataset/0
!unzip -q beit3-fake-image-detector.zip

Saving kaggle.json to kaggle.json
 96% 617M/643M [00:08<00:00, 79.5MB/s]
100% 643M/643M [00:08<00:00, 78.3MB/s]
  0% 0.00/397k [00:00<?, ?B/s]
100% 397k/397k [00:00<00:00, 161MB/s]
 98% 502M/511M [00:06<00:00, 66.7MB/s]
100% 511M/511M [00:06<00:00, 84.8MB/s]


In [ ]:
from copy import copy

import pandas as pd
import torch
import torchmetrics
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint
from timm.loss.binary_cross_entropy import BinaryCrossEntropy
from torch.nn.functional import sigmoid
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


class Model(LightningModule):
    def __init__(self, dataset, predict_dataset, train_transform, eval_transform, batch_size, learning_rate):
        super().__init__()
        self.batch_size = batch_size
        self.learning_rate = learning_rate

        self.beit_wrapper = beit_wrapper

        self.criterion = BinaryCrossEntropy(smoothing=0.1)  # label smoothing
        self.matt = torchmetrics.MatthewsCorrCoef(task='binary')

        train_size = int(0.8 * len(dataset))
        valid_size = len(dataset) - train_size
        self.train_subset, self.val_subset = torch.utils.data.random_split(dataset, [train_size, valid_size])
        self.predict_dataset = predict_dataset

        self.train_subset.dataset = copy(dataset)
        self.train_subset.dataset.transform = train_transform
        self.val_subset.dataset.transform = eval_transform
        self.predict_dataset.transform = eval_transform

    def forward(self, x):
        y_hat = self.beit_wrapper(x)
        return sigmoid(y_hat).squeeze()

    def training_step(self, batch, batch_nb):
        x, y = batch
        y = y.float()
        loss = self.criterion(self(x), y)
        self.log('loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.float()
        y_hat = self(x)
        self.log('val_loss', self.criterion(y_hat, y), on_epoch=True, prog_bar=True, logger=True)
        self.log('val_matt', self.matt(y_hat, y), on_epoch=True, prog_bar=True, logger=True)

    # noinspection PyUnresolvedReferences
    def on_train_epoch_end(self):
        for name, params in self.named_parameters():
            self.logger.experiment.add_histogram(name, params, self.current_epoch)
            if params.grad is not None:
                self.logger.experiment.add_histogram(name + '/grad', params.grad, self.current_epoch)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate, amsgrad=True)

    def train_dataloader(self):
        return DataLoader(self.train_subset, num_workers=4, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_subset, num_workers=4, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.predict_dataset, num_workers=4, batch_size=len(self.predict_dataset))

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, _ = batch
        return self(x)


if __name__ == '__main__':
    seed_everything(42, workers=True)

    artifact_dataset = ImageFolder(root='./artifact_60k_balanced')
    rd_predict_dataset = ImageFolder(root='./rd_predict_dataset')
    model = Model(
        dataset=artifact_dataset,
        predict_dataset=rd_predict_dataset,
        train_transform=beit_train_transform,
        eval_transform=beit_eval_transform,
        batch_size=256,
        learning_rate=1e-4,
    )

    checkpoint_callback = ModelCheckpoint(
        monitor='val_matt',
        mode='max',
        save_top_k=3,
        save_last=True,
    )

    trainer = Trainer(
        accelerator='gpu',
        devices=1,
        max_epochs=600,
        precision=32,
        deterministic=True,
        callbacks=[checkpoint_callback],
    )

    # trainer.fit(model)

    predictions: torch.Tensor = trainer.predict(
        model,
        ckpt_path='epoch114-step19665.ckpt',
    )

    real_mask = (predictions[0] > 0.5).numpy().astype(bool)
    df = pd.DataFrame(rd_predict_dataset.imgs)
    df.loc[real_mask, 1] = 'real'
    df.loc[~real_mask, 1] = 'fake'
    df.to_csv('predictions.csv', index=False, header=False)

INFO:lightning_fabric.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at epoch114-step19665.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at epoch114-step19665.ckpt
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creat

Predicting: 0it [00:00, ?it/s]

In [ ]:
df

,0,1
0,./rd_predict_dataset/0/001.jpg,fake
1,./rd_predict_dataset/0/002.jpg,fake
2,./rd_predict_dataset/0/003.jpg,fake
3,./rd_predict_dataset/0/004.jpg,fake
4,./rd_predict_dataset/0/005.jpg,fake
5,./rd_predict_dataset/0/006.jpg,fake
6,./rd_predict_dataset/0/007.jpg,real
7,./rd_predict_dataset/0/008.png,real
8,./rd_predict_dataset/0/009.jpg,fake
9,./rd_predict_dataset/0/010.jpg,fake
